DCGAN or Deep Convolutional GAN, is a generative adversarial network architecture. It uses a couple of guidelines. In particular:



*   Replacing any pooling layers with the strided convolutions (discriminator) and fractional-strided convolutions (generators)

*   Using bathnorm in both generator and discriminator
*   Removing fully connected hiiden layers for deeper architecture


*   Using ReLU activation in generator for all lyers except for the output which uses tanh


*   Using LeakyReLu activation in the discriminator for all layers

this is the source : https://paperswithcode.com/method/dcgan#:~:text=DCGAN%2C%20or%20Deep%20Convolutional%20GAN,the%20generator%20and%20the%20discriminator.



In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Input, Flatten,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Dropout, Reshape, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_dims = 28
img_chn1 = 1
ltnt_dim = 100

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
img_dims = 28
img_chn1 = 1
ltnt_dim=100
(x_train,y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
x_train = np.expand_dims(x_train,axis=3)
x_test = np.expand_dims(x_test,axis=3)

In [ ]:
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)

x_train shape:  (60000, 28, 28, 1)
x_test shape:  (10000, 28, 28, 1)


In [ ]:
# we will build the generator convolutional neural network
inpt_noise = Input(shape=(ltnt_dim))
g11 = Dense(7*7*256, activation='relu')(inpt_noise)
g12 = BatchNormalization()(g11)
g13 = Reshape((7,7,256))(g12)
g14= Conv2DTranspose(128, (5,5), strides=(1,1),padding='same', activation='relu')(g13)
g15 = BatchNormalization()(g14)
g16= Conv2DTranspose(64, (5,5), strides=(2,2),padding='same', activation='relu')(g15)
g17 = BatchNormalization()(g16)
g18= Conv2DTranspose(1, (5,5), strides=(2,2),padding='same', activation='sigmoid')(g17)
generator = Model(inpt_noise, g18)
gnrtr_img = generator(inpt_noise)
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense (Dense)               (None, 12544)             1266944   
                                                                 
 batch_normalization (BatchN  (None, 12544)            50176     
 ormalization)                                                   
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 7, 7, 128)        819328    
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 128)        512   

In [ ]:
#build the critic convolutional network
inpt_img = Input(shape=(img_dims,img_dims, img_chn1 ))
d11 = Conv2D(64,(5,5), strides=(2,2), padding='same', activation='relu')(inpt_img)
d12= Dropout(0.3)(d11)

d13 = Conv2D(128,(5,5), strides=(2,2), padding='same', activation='relu')(d12)
d14= Dropout(0.3)(d13)

d15 = Flatten()(d14)
d16= Dense(1,activation='sigmoid')(d15)
critic = Model(inpt_img, d16)
validity = critic(gnrtr_img)
critic.summary()
critic.summary() # critic = discriminator

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 64)        1664      
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6273

In [ ]:
optimizer = Adam(0.0002, 0.5)

critic.compile(loss='binary_crossentropy', optimizer=optimizer, 
               metrics=['accuracy'])

critic.trainable = False

gen_crt = Model(inpt_noise, validity)
gen_crt.compile(loss='binary_crossentropy', optimizer=optimizer)

epochs = 12001
batch_size=64
sample_interval=400

In [ ]:
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

samp_imgs = {}
closs = []
gloss = []
cacc = []
for epoch in range(epochs):
  idx = np.random.randint(0, x_train.shape[0], batch_size)
  imgs = x_train[idx]

  noise = np.random.uniform(0, 1, (batch_size, ltnt_dim))
  gen_imgs = generator.predict(noise)
  c_loss_real = critic.train_on_batch(imgs, valid)
  c_loss_fake = critic.train_on_batch(gen_imgs, fake)
  c_loss = 0.5 * np.add(c_loss_real, c_loss_fake)

  noise = np.random.uniform(0, 1, (batch_size, ltnt_dim))
  g_loss = gen_crt.train_on_batch(noise, valid)
  
  closs.append(c_loss[0])
  cacc.append(c_loss[1])
  gloss.append(g_loss)

  if epoch % sample_interval == 0:
    print ("%d [C loss: %f, acc.: %.2f%%] [G loss: %f]" % 
           (epoch, c_loss[0], 100*c_loss[1], g_loss))
  
    noise = np.random.uniform(0, 1, (2, ltnt_dim))
    gen_imgs = generator.predict(noise)
    samp_imgs[epoch] = gen_imgs

0 [C loss: 0.710588, acc.: 21.88%] [G loss: 0.687957]
400 [C loss: 0.000046, acc.: 100.00%] [G loss: 0.000028]
800 [C loss: 0.025624, acc.: 99.22%] [G loss: 0.012787]
1200 [C loss: 0.711656, acc.: 50.00%] [G loss: 0.392552]
1600 [C loss: 0.630438, acc.: 65.62%] [G loss: 0.795434]
2000 [C loss: 0.620446, acc.: 63.28%] [G loss: 0.627613]
2400 [C loss: 0.597491, acc.: 65.62%] [G loss: 0.933187]
2800 [C loss: 0.638771, acc.: 58.59%] [G loss: 0.716047]
3200 [C loss: 0.664199, acc.: 55.47%] [G loss: 0.830691]
3600 [C loss: 0.669791, acc.: 63.28%] [G loss: 0.789071]
4000 [C loss: 0.674077, acc.: 58.59%] [G loss: 0.748519]
4400 [C loss: 0.678821, acc.: 60.94%] [G loss: 0.740943]
4800 [C loss: 0.685091, acc.: 56.25%] [G loss: 0.740619]
5200 [C loss: 0.681371, acc.: 56.25%] [G loss: 0.740327]
5600 [C loss: 0.681140, acc.: 57.03%] [G loss: 0.727129]
6000 [C loss: 0.691874, acc.: 53.91%] [G loss: 0.713781]
6400 [C loss: 0.677927, acc.: 62.50%] [G loss: 0.731661]
6800 [C loss: 0.675277, acc.: 60.94

The above result can  look different in your laptop or computer because this is all based on random noise. This randomness aspect will take your model in a different direction. However,  what you will see is that your generator's loss will decrease dradually and if the generator works properly, the accuracy should be geetting closer to random change, that is close to 50%. If your discriminator is always 100% then your generator is not good enough, and if your dicriminator is around 50% accuracy, then your generator might be too good or discrimitor is too weak.